In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



import bs4
import requests
import re



url = 'https://www.gutenberg.org/ebooks/subject/1415'
res = requests.get(url)
soup = bs4.BeautifulSoup(res.text, 'html.parser')
element = soup.select('a')
books = []
for i in element:
    link = i.get('href')
    if (link[:8] == '/ebooks/') & (link[8:].isdigit()):
        books.append(link)
books
len(books)



for idx in range(26,451,25):
    url = 'https://www.gutenberg.org/ebooks/subject/1415?start_index=' + str(idx)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    element = soup.select('a')
    for i in element:
        link = i.get('href')
        if (link[:8] == '/ebooks/') & (link[8:].isdigit()):
            books.append(link)
len(books)



audio = ['19573','23716','23718','28941','21150','9539','22435','19788']

for elem in books:
    if elem[8:] in audio:
        books.remove(elem)
len(books)



files = []
books1 = []

for b in books:
    url = 'https://www.gutenberg.org/' + str(b)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    element = soup.select('a')
    for i in element:
        link = str(i.get('href'))
        if link.find('.txt') != -1:
            files.append(link)
            books1.append(b)
len(files)
len(books1)



files_local = files.copy()
books_local = books1.copy()
books = books1.copy()


nb = len(files_local)
idx_bad = []
for i in range(nb):
    url ='https:' + str(files_local[i])
    res = requests.get(url)
    regex = re.compile(r'Language:\s\w*')
    lang = regex.findall(res.text)
    if lang[0].upper() != 'Language: English'.upper():
        print(i)
        idx_bad.append(i)



files_eng = []
books_eng = []
for i in range(nb):
    if i not in idx_bad:
        files_eng.append(files_local[i])
        books_eng.append(books_local[i])



len(files_eng)
len(books_eng)
len(idx_bad)


from urllib.parse import unquote
nb = len(books_eng)
titles = []

for j in range(nb):
    if j % 10 == 0:
        print(j)
    url = 'https://www.gutenberg.org' + str(books_eng[j])
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    element = soup.select('a')
    for i in element:
        link = str(i.get('href'))
        if link.find('https://twitter.com/share?text=') != -1:
            titles.append(unquote(link[len('https://twitter.com/share?text='):link.find('&url=')]))




len(titles)
len(set(titles))


files_unq = []
books_unq = []
titles_unq = []

for i in range(len(titles)):
    if titles[i] not in titles_unq:
        titles_unq.append(titles[i])
        books_unq.append(books_eng[i])
        files_unq.append(files_eng[i])
        
len(files_unq)
len(books_unq)
len(titles_unq)



titles_unq[0]
books_unq[0]
files_unq[0]
titles_unq[:5]


content_unq = []
nu = len(files_unq)

for i in range(nu):
    url ='https:' + str(files_unq[i])
    res = requests.get(url)
    content_unq.append(res.text)
    if i % 10 == 0:
        print(i)
    



content_unq[3]



with open('files_unq.txt', 'w') as f:
    for item in files_unq:
        f.write("%s\n" % item)



with open('books_unq.txt', 'w') as f:
    for item in books_unq:
        f.write("%s\n" % item)



with open('titles_unq.txt', 'w') as f:
    for item in titles_unq:
        f.write("%s\n" % item)



# load additional module
import pickle

with open('content_unq.data', 'wb') as filehandle:  
    # store the data as binary data stream
    pickle.dump(content_unq, filehandle)



####################### Post extraction content files and links ###########################

files_unq = open("files_unq.txt", "r").read().split()
print(files_unq)



import pickle
with open('content_unq.data', 'rb') as filehandle:  
    # read the data as binary data stream
    content_unq = pickle.load(filehandle)



tokens = []
for c in content_unq:
    tokens.append(list(filter(None, re.sub(r"\r", "`", re.sub(r"\n", "`", c)).split('`'))))



y = 0
z = 0
extra = []

for i in range(len(tokens)):
    alpha = 0
    omega = 0
    for line in tokens[i]:
        if re.sub(r" ","",line)[:10] == '***STARTOF':
            y += 1
            alpha += 1
            print(line)
        if line[:6] == 'End of':
            z += 1
            omega += 1
            print(line)
    if omega >= 3:
        extra.append(i)
    print("\n")
y/len(tokens)
z/len(tokens)
extra



for i in range(len(tokens)):
    for j in range(len(tokens[i])):
        if re.sub(r" ","",tokens[i][j])[:10] == '***STARTOF':
            alpha = j+1
        if tokens[i][j][:6] == 'End of':
            omega = j
            break
        if re.sub(r" ","",tokens[i][j])[:8] == '***ENDOF':
            omega = j
            break
    tokens[i] = tokens[i][alpha:omega]




for t in tokens:
    print(t[-1])



# load additional module
import pickle

with open('tokens.data', 'wb') as filehandle:  
    # store the data as binary data stream
    pickle.dump(tokens, filehandle)



##################### Post tokenization #####################3
import pickle
with open('tokens.data', 'rb') as filehandle:  
    # read the data as binary data stream
    tokens = pickle.load(filehandle)




tokens[8][-20:]

